In [1]:
!pip install azureml
!pip install azureml.core
!pip install azureml.widgets
!pip install azureml.pipeline

  Using cached azureml_core-1.43.0-py3-none-any.whl (2.7 MB)
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-automl-core~=1.42.0, but you'll have azureml-automl-core 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-core~=1.42.0, but you'll have azureml-core 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-dataset-runtime[fuse,pandas]~=1.42.0, but you'll have azureml-dataset-runtime 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-telemetry~=1.42.0, but you'll have azureml-telemetry 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-train-automl-client~=1.42.0, but you'll have azureml-train-automl-client 1.43.0 which is incompatible.
ERROR: azureml-tensorboard 1.42.0 has requirement azureml-core~=1.42.0, but you'll have azureml-core 1.43.0 which is incompatible.
ERROR: azureml-sdk 

In [2]:
import os
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset, Run
from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [3]:
ws = Workspace.from_config()

In [4]:
keyvault = ws.get_default_keyvault()

In [5]:
default_datastore = ws.get_default_datastore()

### Create CPU Compute

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_cluster_name = 'gpunc6'
vm_size = 'STANDARD_NC6'
#vm_size = 'STANDARD_NC12S_V3'
# Check cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print('Found existing cluster {}'.format(aml_compute_cluster_name))
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=6)
    aml_compute = ComputeTarget.create(ws, aml_compute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster gpunc6
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
# https://docs.microsoft.com/en-us/azure/container-registry/container-registry-auth-service-principal

from azureml.core.runconfig import RunConfiguration
aml_run_config = RunConfiguration()

from azureml.core.environment import Environment
# Create the environment
env = Environment(name='rp')
# Enable Docker and reference an image
env.docker.enabled = True
env.python.user_managed_dependencies=True

env.docker.base_image_registry.address = "csssdemocr.azurecr.io"
env.docker.base_image_registry.username = keyvault.get_secret(name='csssdemocr-admin-user')
env.docker.base_image_registry.password = keyvault.get_secret(name='csssdemocr-admin-pwd')
env.docker.base_image = "csssdemocr.azurecr.io/azureml-env-base-sd:latest"
                          
aml_run_config.environment = env
env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "csssdemocr.azurecr.io/azureml-env-base-sd:latest",
        "baseImageRegistry": {
            "address": "csssdemocr.azurecr.io",
            "password": "AzureMlSecret=Env_796e80c7e5bf8a48cb603f229eeec578ec72443dbfe37710cc80de67228c6713_25#EnvironmentDefinition#ContainerRegistry#Password",
            "registryIdentity": null,
            "username": "csssdemocr"
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "rp",
    "python": {
        "baseCondaEn

### Define Runtime Configuration

## Specify script source folder(s)

In [8]:
model_script_folder = ''

print('Source directory for model is {}.'.format(os.path.realpath(model_script_folder)))

Source directory for model is /mnt/batch/tasks/shared/LS_root/mounts/clusters/cpu-std-ds11v2/code/Users/jp/search-demo/model/cord19.


### Specify Working Datastore

In [9]:
embed_wrkdir = PipelineData('wrkdir', datastore=default_datastore)

### Step - Model Embed

In [10]:
dataset_size_percent = 1
step_model_embed_corpus = PythonScriptStep(
    name='Model Embed Corpus',
    script_name='step_model_embed_corpus.py', 
    arguments=['--dataset_size_percent', dataset_size_percent,
               '--wrkdir', embed_wrkdir,
               '--neural_model_name', 'c19_ance_msmarco_passage'],
    outputs=[embed_wrkdir],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=model_script_folder,
    allow_reuse=True
)

print('Step Model Embed Corpus Created')

Step Model Embed Corpus Created


### Step - Corpus to ElasticSearch

In [11]:
step_model_corpus_to_es = PythonScriptStep(
    name='Model Corpus To ES',
    script_name='step_model_corpus_to_es.py', 
    arguments=['--wrkdir', embed_wrkdir,
               '--es_batch_chunk_size', 50],
    inputs=[embed_wrkdir],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=model_script_folder,
    allow_reuse=True
)

print('Step Model Corpus to ElasticSearch Created')

Step Model Corpus to ElasticSearch Created


### Step - Model Deploy

In [12]:
#step_model_deploy = PythonScriptStep(
#    name='Model Deploy',
#    script_name='step_model_deploy.py', 
#    arguments=['--wrkdir', embed_wrkdir,
#               '--no-deploy'],
#    inputs=[embed_wrkdir],
#    compute_target=aml_compute,
#    runconfig=aml_run_config,
#    source_directory=model_script_folder,
#    allow_reuse=True
#)

#print('Step Model Deploy Created')

### Create Step List

In [13]:
#steps = [step_model_embed_corpus, step_model_deploy]
steps = [step_model_embed_corpus, step_model_corpus_to_es]
print("Step lists created")

Step lists created


In [14]:
aml_pipeline = Pipeline(workspace=ws, steps=steps)
print ('Pipeline is built')

Pipeline is built


In [15]:
aml_pipeline.validate()
print('Pipeline validation complete')

Step Model Embed Corpus is ready to be created [438cadb0]
Step Model Corpus To ES is ready to be created [ea2d8115]
Pipeline validation complete


In [16]:
# Note regenerate output, decides whether to execute the module to regenerate output at running time.
# It's by default unselected, which means if the module has been executed with the same parameters previously, 
# the system will reuse the output from last run to reduce run time.

aml_pipeline_run = Experiment(ws, 'pipeline-cord19-corpus-to-es').submit(aml_pipeline, regenerate_outputs=True)
print('Experiment pipeline cord19 corpus to ElasticSearch is submitted for execution')

Created step Model Embed Corpus [438cadb0][a0bf98ca-973e-4c79-87ba-e9a137ec2d47], (This step will run and generate new outputs)
Created step Model Corpus To ES [ea2d8115][3938a057-4016-40f1-a79e-dbc9826b1cfb], (This step will run and generate new outputs)
Submitted PipelineRun d7990cfc-8f1c-4c50-91b0-1f19852f5fff
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d7990cfc-8f1c-4c50-91b0-1f19852f5fff?wsid=/subscriptions/fad2758e-cc1a-4f45-8323-6b466001d50a/resourcegroups/cs-semantic-search-demo-rg/workspaces/cs-semantic-search-demo-mlw&tid=67119780-27b3-4331-813a-0b239d67524a
Experiment pipeline cord19 corpus to ElasticSearch is submitted for execution


In [17]:
RunDetails(aml_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [18]:
step_runs = aml_pipeline_run.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)